# Ergonomic Error Handling In Rust

Returning a `Result<T, E>` works great when there is a single error type E. But things become more complicated when we want to work with multiple error types.

Let’s look at a small example that covers the easy case of a single error type. We’ll try to open a file that does not exist.

In [26]:
use std::fs::File;

let _f = File::open("invisible.txt")?;

No such file or directory (os error 2)


Now let’s introduce another error type in the same code.

In [24]:
use std::net::Ipv6Addr;

fn dummy_main() -> Result<(), std::io::Error> {
    let _f = File::open("invisible.txt")?;

    let _localhost = "::1".parse::<Ipv6Addr>()?;  

    Ok(())  
}

dummy_main()

Error: `?` couldn't convert the error to `std::io::Error`

Why are there multiple messages about `std::convert::From` ? Well, the `?` operator is syntactic sugar for the `try!` macro. `try!` performs two functions:

1. When it detects `Ok(value)`, the expression evaluates to value.
2.  When `Err(err)` occurs, `try!/?` returns early after attempting to convert `err` to the `error` type defined in the calling function.

In the above code block  we can see the try! macro in action as:

1. `File::open()` returns `std::io::Error`, so no conversion is necessary.
2. `.parse()` presents `?` with a `std::net::AddrParseError`. We don’t define how to convert `std::net::AddrParseError`
to `std::io::Error`, so the program fails to compile.

Because the signature of main is `main() ? Result<(), std::io ::Error>`, Rust attempts to convert the `std::net::AddrParseError` produced by `parse::<Ipv6Addr>()` into a `std::io::Error`.

We can fix this error by using `Box<dyn Error>` as the error variant. The `dyn` keyword is short for dynamic, implying that there is a runtime cost for this flexibility.

In [23]:
use std::error::Error;

fn dummy_main() -> Result<(), Box<dyn Error>> {
    let _f = File::open("invisible.txt")?;

    let _localhost = "::1".parse::<Ipv6Addr>()?;  

    Ok(())  
}

dummy_main()

Err(Os { code: 2, kind: NotFound, message: "No such file or directory" })

We have managed to fix the error. By implementing a trait object in a return value to simplify error handling when errors originate from multiple upstream crates.

Wrapping trait objects in `Box` is necessary because their size (in bytes on the stack) is unknown at compile time. The trait object might originate from either `File::open()` or `"::1".parse()`. What actually happens depends on the circumstances encountered at runtime.

## Defining Our Own Error Type

Different dependencies define their own error types. Multiple error types in one function prevent returning `Result`. We can use trait objects, but trait objects have a potentially significant downside. Using trait objects is also known as **type erasure**. Rust is no longer aware that an error has originated upstream. Using `Box<dyn Error>` as the error variant of a Result means that the upstream error types are, in a sense, lost as the original errors are now converted to exactly the same type.

It is possible to retain the upstream errors by bundling upstream errors in our own type. When the upstream errors are needed later (say, for reporting errors to the user), it’s possible to extract these with pattern matching. We can do so by:

1. Define an enum that includes the upstream errors as variants.
2. Annotate the enum with `#[derive(Debug)]`.
3. Implement Display.
4. Implement Error, which almost comes for free because we have implemented Debug and Display.
5. Use `map_err()` in your code to convert the upstream error to your omnibus error type.

There’s an optional extra step we can implement that improves the ergonomics. We need to implement `std::convert::From` to remove the need to call `map_err()`.

In [16]:
// define an enum that includes the upstream errors as variants
use std::io;
use std::net;

#[derive(Debug)] 
enum UpstreamError {
    IO(io::Error),
    Parsing(net::AddrParseError)
}

In [18]:
use std::fmt;

// implementing std::fmt::display
impl fmt::Display for UpstreamError {
    fn fmt(&self, f: &mut fmt::Formatter<'_>) -> fmt::Result {
        write!(f, "{:?}", self) // implements Display in terms of Debug via the "{:?}" syntax
    }
}

In [19]:
// implement std::error::Error
use std::error;

impl error::Error for UpstreamError { } // Defers to default method implementations. The compiler will fill in the blanks.

In [21]:
// using map_err()
fn dummy_main() -> Result<(), UpstreamError> {
    let _f = File::open("invisible.txt")
      .map_err(UpstreamError::IO)?;
    
    let _localhost = "::1".parse::<Ipv6Addr>()
      .map_err(UpstreamError::Parsing)?;

    Ok(()) 
}

In [22]:
dummy_main()

Err(IO(Os { code: 2, kind: NotFound, message: "No such file or directory" }))

The code compiles!. The `map_err()` function maps an error to a function. Variants of our UpstreamError enum can be used as functions here.  Note that the `?` operator needs to be at the end. Otherwise, the function can return before the code has a chance to convert the error.